In [7]:
import sys
sys.path.append('/home/pranavsatheesh/host_galaxies/')
import illustris_python as il
import matplotlib.pyplot as plt
import h5py
import numpy as np
from scipy.spatial import cKDTree

In [3]:
basepath = '/orange/lblecha/IllustrisTNG/Runs/TNG50-1/output'
merger_file_path = '/home/pranavsatheesh/host_galaxies/data/merger_files'

In [135]:
snap = 50
subhalos = il.groupcat.loadSubhalos(
                basepath, snap, 
                fields=['SubhaloLenType', 'SubhaloMassType', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloSFR','SubhaloPos','SubhaloHalfmassRadType']
            )

header = il.groupcat.loadHeader(basepath, snap)
h =header['HubbleParam']
box_size = header['BoxSize']/h
z = header['Redshift']
print(header['Redshift'])
subhalo_pos = subhalos['SubhaloPos']/h 

0.9972942257819404


In [137]:
subhalo_ids = np.arange(0,len(subhalos['SubhaloPos']))
print(len(subhalo_ids))

6780233


In [138]:
Ngas = subhalos['SubhaloLenType'][:, 0]
Ndm = subhalos['SubhaloLenType'][:, 1]
Nstar = subhalos['SubhaloLenType'][:, 4]
Nbh = subhalos['SubhaloLenType'][:, 5]

valid_mask = (Ngas >= 100) & (Ndm >= 100) & (Nstar >= 100) & (Nbh >= 0)
print(valid_mask)
subhalo_ids = subhalo_ids[valid_mask]
subhalo_pos = subhalo_pos[valid_mask]
print(len(subhalo_ids))

[ True  True  True ... False False False]
20163


In [139]:
tree = cKDTree(subhalo_pos, boxsize=box_size)
distances, nearest_neighbors = tree.query(subhalo_pos,k=2)

nearest_subhalo_ids = nearest_neighbors[:,-1]
r_subhalos = distances[:,-1]

stellar_half_mass_radius = subhalos['SubhaloHalfmassRadType'][:,4]/h


r_sep = r_subhalos/(stellar_half_mass_radius[subhalo_ids] + stellar_half_mass_radius[nearest_subhalo_ids] )


valid_mask = r_sep>2 #only including non interacting galaxies with a closest companinon relative seperation above 2

subhalo_ids = subhalo_ids[valid_mask]
print(len(subhalo_ids))

20056
